In [1]:
from google.colab import drive
drive.mount('IEEE')
import os
 
# 此处为google drive中的文件路径,drive为之前指定的工作根目录，要加上
os.chdir("IEEE/My Drive/IEEE")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at IEEE


In [ ]:
import os
 
# 此处为google drive中的文件路径,drive为之前指定的工作根目录，要加上
os.chdir("IEEE/My Drive/IEEE")

FileNotFoundError: ignored

In [ ]:
#授权绑定下面的代码
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate


OSError: ignored

In [2]:
pip install transformers==2.8.0 pandas gensim scikit-learn filelock gdown

     |████████████████████████████████| 573kB 8.3MB/s 
     |████████████████████████████████| 3.7MB 22.7MB/s 
     |████████████████████████████████| 1.1MB 57.5MB/s 
     |████████████████████████████████| 890kB 58.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=34eb25badec4fa63d516aa94d9160140564f0ce68d21c0dbaf08365f4b8103e1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
%matplotlib inline
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
import datetime
import pickle
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import os
import torch
middlef='middle/'

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
milestones=pd.read_csv("data/IBI_case_milestones_anonymized.csv")
case_status=pd.read_csv("data/IBI_case_status_history_v2.csv")
test_cases=pd.read_csv("data/IBI_test_cases_no_target.csv")
del test_cases['INV_TIME_TO_NEXT_ESCALATION']
test=pd.merge(test_cases, case_status, on=['REFERENCEID', 'SECONDS_SINCE_CASE_START'])
caset=case_status[~case_status['INV_TIME_TO_NEXT_ESCALATION'].isnull()]
#把重大事件表划分为测试和训练两部分
milestones_train=milestones[milestones['REFERENCEID'].isin(caset['REFERENCEID'])]
milestones_test=milestones[milestones['REFERENCEID'].isin(test['REFERENCEID'])]

In [ ]:
from sklearn.feature_extraction.text import  TfidfVectorizer
import pickle

from sklearn.decomposition import TruncatedSVD
def getmilevector(df,istrain,needfit,n_fea):
    """
    塞进来一个包含短文本的dataframe,把每个文本转换成n_fea维的向量，
    istrain决定是否训练，needfit为包含短文本描述的列
    返回一个dataframe，多出来了文本得到的向量
    """
    feature=[]
    for col in needfit:
        doc=df[col]
        
        if istrain:
            tfidf_vec = TfidfVectorizer(ngram_range=(1,2),analyzer='word',smooth_idf=1)
            discuss_tf  = tfidf_vec.fit_transform(doc)
            pickle.dump(tfidf_vec, open(col+".pickle", "wb"))

            svd_tag_tmp = TruncatedSVD(n_components=n_fea, n_iter=20, random_state=2019)
            tag_svd_tmp = svd_tag_tmp.fit_transform(discuss_tf )
            pickle.dump(svd_tag_tmp, open(col+"svd_tag_tmp.pickle", "wb"))
        else:
            tfidf_vec = pickle.load(open(col+".pickle" ,"rb"))
            discuss_tf  =tfidf_vec.transform(doc)

            svd_tag_tmp = pickle.load(open(col+"svd_tag_tmp.pickle", "rb"))
            tag_svd_tmp = svd_tag_tmp.transform(discuss_tf)
        tag_svd_tmp = pd.DataFrame(tag_svd_tmp)
        tag_svd_tmp.columns = [col+f'b_svd_{i}' for i in range(n_fea)]    
        feature.append(tag_svd_tmp)
    vecfeature=pd.concat(feature,axis=1)
    df1=df.reset_index()
    df2=pd.concat([df1,vecfeature],axis=1)
    return df2

mt=getmilevector(milestones_train,True,['MILESTONEDESCRIPTION','NOTEDESCRIPTION'],40)
mtest=getmilevector(milestones_test,False,['MILESTONEDESCRIPTION','NOTEDESCRIPTION'],40)
def changeOthertodigital(df,istrain):
    del df['index']
    del df['MILESTONEDESCRIPTION']
    del df['NOTEDESCRIPTION']
    df['UPDATED_BY']=df['UPDATED_BY'].astype('category')
    df['CREATED_BY']=df['CREATED_BY'].astype('category')
    if istrain:
        usrlist=list(set(list(milestones['UPDATED_BY'])+list(milestones['CREATED_BY'])))
        usermap={v:i for i,v in enumerate(usrlist)}
        pickle.dump(usermap, open("usermap.pickle", "wb"))
    else:
        usermap = pickle.load(open("usermap.pickle", "rb"))
    
    df['UPDATED_BY']=df['UPDATED_BY'].map(usermap)
    df['CREATED_BY']=df['CREATED_BY'].map(usermap)
    df=df.rename(columns = {"SECONDS_SINCE_CASE_START": "milestone_SECONDS_SINCE_CASE_START"})
    df=df.rename(columns = {"CREATED_BY": "milestone_CREATED_BY"})
    df=df.rename(columns = {"UPDATED_BY": "milestone_UPDATED_BY"})
    return df
mt=changeOthertodigital(mt,True)
mtest=changeOthertodigital(mtest,False)

mt.to_csv(middlef+"miilestone_fortrain_digitle.csv",index=False)
mtest.to_csv(middlef+"miilestone_fortest_digitle.csv",index=False)

In [4]:
!pip install pandarallel

  Created wheel for pandarallel: filename=pandarallel-1.5.1-cp36-none-any.whl size=17126 sha256=c512c5e8b9dec3e5e36493414254a98968bfa4739058cf855269ad493e1e3b37
  Stored in directory: /root/.cache/pip/wheels/6f/c8/e5/d43fa63105ce1dd22f4df51bc2edfefd54d92ce64f25326314
Successfully built pandarallel


In [ ]:
mt=pd.read_csv(middlef+"miilestone_fortrain_digitle.csv")
mtest=pd.read_csv(middlef+"miilestone_fortest_digitle.csv")

In [ ]:
from pandarallel import pandarallel
pandarallel.initialize()

def getsquen(x,tmt,columsbase):
    caseID=x['REFERENCEID']
    escalatedTime=x['SECONDS_SINCE_CASE_START']
    if caseID in tmt:
        milestonsession=tmt[caseID]
    else:
        milestonsession=pd.DataFrame(columns=columsbase)
    #     print(type(escalatedTime))
    milestonsession['neartime']=escalatedTime-milestonsession['milestone_SECONDS_SINCE_CASE_START']
    milestonsession=milestonsession[milestonsession['neartime']>=0 ]#position embeding用
    #拿些统计特征 序列 size,milestoneID unique,时间的unique，事件次数的sum,mean,std
    sqe_size=len(milestonsession)
    miles_unique=milestonsession['MILESTONEID'].nunique()
    time_unique=milestonsession['milestone_SECONDS_SINCE_CASE_START'].nunique()
    tmp=milestonsession['MILESTONEID'].value_counts()
    milestone_times_sum=tmp.sum()
    milestone_times_mean=tmp.mean()
    milestone_times_std=tmp.std()
    #拿到相应的时序序列，以及时间序列
    milestone_seq=' '.join(list(milestonsession['MILESTONEID'].apply(str)))
    time_seq=' '.join(list(milestonsession['neartime'].apply(str)))
    return pd.Series({'REFERENCEID':caseID,'SECONDS_SINCE_CASE_START':escalatedTime,'milestone_seq':milestone_seq,'time_seq':time_seq,
                      'CREATED_BY_seq':' '.join(list(milestonsession['milestone_CREATED_BY'].apply(str))),'UPDATED_BY_seq':' '.join(list(milestonsession['milestone_UPDATED_BY'].apply(str))),
                   'milestones_size':sqe_size,'milestones_unique':miles_unique,'time_unique':time_unique,
                    'milestone_times_sum':milestone_times_sum,'milestone_times_mean':milestone_times_mean,'milestone_times_std':milestone_times_std})
     
tmt={name:group for name, group in mt.groupby(mt['REFERENCEID'])}
train_featurelist=caset.parallel_apply(lambda x:getsquen(x,tmt,mt.columns),axis=1)
tmtest={name:group for name, group in mtest.groupby(mtest['REFERENCEID'])}
test_featurelist=test.parallel_apply(lambda x:getsquen(x,tmtest,mtest.columns),axis=1)

################################################################################
print("Extract features done! saving data...")
train_featurelist.to_pickle(middlef+'train_milestone_sque.pkl')#dataframe for train
test_featurelist.to_pickle(middlef+'test_milestone_sque.pkl')#dataframe for test

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
!zip -r a1.zip  middle/train_milestone_sque.pkl

  adding: middle/train_milestone_sque.pkl (deflated 82%)


In [ ]:
train_df=pd.read_pickle('middle/train_milestone_sque.pkl')#这里面是序列化的数据
test_df=pd.read_pickle('middle/test_milestone_sque.pkl')
mytrain=pd.read_csv("middle/train.csv")#这里面是最原始的按referrenceId和time做主键的数据
mytest=pd.read_csv("middle/test.csv")
del mytrain['Unnamed: 0']
del mytest['Unnamed: 0']
allfeature=list(mytrain.columns)
allfeature.remove('INV_TIME_TO_NEXT_ESCALATION')
allfeature.remove('CLOUD')

newtrain=pd.read_csv("middle/train_data.csv")#这里面是最原始的按referrenceId做主键,按baseline处理得到的数据
newtest=pd.read_csv("middle/test_data.csv")
res_ids=list(newtrain['REFERENCEID'].unique())
test_ids=list(newtest['REFERENCEID'].unique())

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:

with open('middle/densefeature.pkl', 'wb') as file:
    pickle.dump(allfeature, file)

In [ ]:
test_df[test_df['milestones_size']!=0]

,REFERENCEID,SECONDS_SINCE_CASE_START,milestone_seq,time_seq,CREATED_BY_seq,UPDATED_BY_seq,milestones_size,milestones_unique,time_unique,milestone_times_sum,milestone_times_mean,milestone_times_std
8467,137780,0,,,,,0,0,0,0,NaN,NaN


In [ ]:
train_df.head(2)

,REFERENCEID,SECONDS_SINCE_CASE_START,milestone_seq,time_seq,CREATED_BY_seq,UPDATED_BY_seq,milestones_size,milestones_unique,time_unique,milestone_times_sum,milestone_times_mean,milestone_times_std
0,100001,0.0,1 12,0.0 0.0,54 54,54 54,2,2,1,2,1.0,0.0
1,100001,2114.0,1 12 36 35,2114.0 2114.0 0.0 0.0,54 54 433 433,54 54 433 433,4,4,2,4,1.0,0.0


In [ ]:
alldata=mytrain.append(mytest)
from sklearn.preprocessing import LabelEncoder
for name,dtype in mytrain.dtypes.iteritems():
  try:
    if dtype=='object':
      encoder = LabelEncoder().fit(alldata[name])   
      mytrain[name]=encoder.transform(mytrain[name])
      # print(mytrain[name])
      mytest[name]=encoder.transform(mytest[name])
  except:
    alldata[name]=alldata[name].astype(str)
    mytrain[name]=mytrain[name].astype(str)
    mytest[name]=mytest[name].astype(str)
    encoder = LabelEncoder().fit(alldata[name])
    mytrain[name]=encoder.transform(mytrain[name])
    mytest[name]=encoder.transform(mytest[name])

mytrain=mytrain.dropna(subset=['milestone_SECONDS_SINCE_CASE_START'])
mytrain=mytrain.dropna(subset=['comments_SECONDS_SINCE_CASE_START'])
alltrain=pd.merge(mytrain,train_df,on=['REFERENCEID','SECONDS_SINCE_CASE_START'])
alltest=pd.merge(mytest,test_df,on=['REFERENCEID','SECONDS_SINCE_CASE_START'])
alltrain.to_pickle(middlef+'train_all.pkl')#dataframe for train
alltest.to_pickle(middlef+'test_all.pkl')#dataframe for test

In [ ]:
needcolums=[]
for c in allfeature:
  if alltrain[c].isnull().any():
    needcolums.append(c)

PRIMARYPRODUCTOSPLATFORMID


In [ ]:
!zip -r a1.zip  middle/train_all.pkl

updating: middle/train_all.pkl (deflated 91%)


In [ ]:
!unzip Bert.zip

Archive:  Bert.zip
   creating: Bert/__pycache__/
  inflating: Bert/__pycache__/model.cpython-37.pyc  
  inflating: Bert/model.py           
  inflating: Bert/run.py             
   creating: Bert/saved_models/


In [ ]:
alltrain.head(2)

,REFERENCEID,SITECODE,SITENAME,CONTACTCOMPANY,COMPANYID,COMPANYOID,INITIALUSERGROUPID,INITIALUSERGROUPDESC,MEDPROJECTID,MEDPROJECTAREA,MEDPROJOPENBY,IWAYJIRAISSUEID,NEWIWAYJIRAISSUEID,PREMIUMCODE,PRIMARYPRODUCTFAMILYID,PRIMARYPRODUCTFAMILYDESC,PRIMARYPRODUCTID,PRIMARYPRODUCTDESC,PRIMARYRELEASEID,PRIMARYRELEASEDESC,PRIMARYPRODUCTAREAID,PRIMARYPRODUCTAREADESC,PRIMARYPRODUCTOSFAMILYID,PRIMARYPRODUCTOSFAMILYDESC,PRIMARYPRODUCTOSPLATFORMID,PRIMARYPRODUCTOSPLATFORMDESC,CONTACTMETHODFLAG,CUSTOMERFIRST,CUSTOMERMIDDLE,CUSTOMERLAST,CUSTOMERNAME,CONTACTPHONE,CONTACTMOBILEPHONE,CONTACTEMAIL,IBICUSTOMERFIRST,IBICUSTOMERMI,IBICUSTOMERLAST,IBICUSTOMERNAME,IBIPHONE,IBIMOBILE,IBIEMAIL,CUSTOMERLABEL,BRANCHCODE,COUNTRY,AGENT_ID,AGENT_NAME,ASM_NAME,SITECOMPANYNAME,SITECOMPANY_OID,SITECOMPANYID,PRIMARYPRODUCTVERSION,CLOUD,CASENUM,PROJNUM,IWAYJIRA,PNOTARGET,PNOCRITICALUSER,ISPREMIUM,CUSTOMER_NAME,CUSTOMER_LABEL,GLOBAL_ID,CUSTOMER_PHASE,P1PHONE,SITEINSTALLYEAR,SECONDS_SINCE_CASE_START,SEVERITY,ISESCALATE,INV_TIME_TO_NEXT_ESCALATION,formerbugtimes,formercomtimes,milestone_seq,time_seq,CREATED_BY_seq,UPDATED_BY_seq,milestones_size,milestones_unique,time_unique,milestone_times_sum,milestone_times_mean,milestone_times_std
0,100001,1302,935,973,932,595,34,10,653,37,16,8,31,0,3,3,34,1,39,78,89,72,1,5,169.0,32,1,1664,51,461,1454,1423,278,284,44,2,48,44,49,30,45,1199,72,39,3,12,28,853,506,114,97,NaN,31295,2839,172,36,45,0,1583,10096,61,3,1532,2013,0.0,2,0,0.0,2.0,1.0,1 12,0.0 0.0,54 54,54 54,2,2,1,2,1.0,0.0
1,100001,1302,935,973,932,595,34,10,653,37,16,8,31,0,3,3,34,1,39,78,89,72,1,5,169.0,32,1,1664,51,461,1454,1423,278,284,44,2,48,44,49,30,45,1199,72,39,3,12,28,853,506,114,97,NaN,31295,2839,172,36,45,0,1583,10096,61,3,1532,2013,2114.0,2,0,0.0,4.0,2.0,1 12 36 35,2114.0 2114.0 0.0 0.0,54 54 433 433,54 54 433 433,4,4,2,4,1.0,0.0


In [ ]:
alltrain['nescated']=alltrain.parallel_apply(lambda x: 0 if x['INV_TIME_TO_NEXT_ESCALATION']==0 else 1,axis=1)

In [ ]:
import os
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from tqdm import tqdm
import pickle
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

def w2v(dfs,f,L=128):
    print("w2v",f)
    sentences=[]
    for df in dfs:
        for line in df[f].values:
            sentences.append(line.split())
    print("Sentence Num {}".format(len(sentences)))
    w2v=Word2Vec(sentences,size=L, window=5,min_count=1,sg=1,workers=16,iter=10)
    print("save w2v to {}".format(os.path.join('word2vecf',f+".{}d".format(L))))
    pickle.dump(w2v,open(os.path.join('word2vecf',f+".{}d".format(L)),'wb'))  

if __name__ == "__main__":
    train_df=pd.read_pickle('middle/train_all.pkl')
    test_df=pd.read_pickle('middle/test_all.pkl')
    #训练word2vector，维度为128
    w2v([train_df,test_df],'milestone_seq',L=128)
    w2v([train_df,test_df],'CREATED_BY_seq',L=128)
    w2v([train_df,test_df],'UPDATED_BY_seq',L=128)

w2v milestone_seq


08/21/2020 08:22:07 - INFO - gensim.models.word2vec -   collecting all words and their counts
08/21/2020 08:22:07 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
08/21/2020 08:22:07 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #10000, processed 253166 words, keeping 58 word types
08/21/2020 08:22:07 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #20000, processed 463255 words, keeping 59 word types
08/21/2020 08:22:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #30000, processed 654538 words, keeping 62 word types
08/21/2020 08:22:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #40000, processed 859787 words, keeping 62 word types
08/21/2020 08:22:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #50000, processed 1090795 words, keeping 62 word types
08/21/2020 08:22:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #60000, processed 1300595 words, keeping 63 w

Sentence Num 720924


08/21/2020 08:22:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #70000, processed 1495792 words, keeping 64 word types
08/21/2020 08:22:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #80000, processed 1691255 words, keeping 64 word types
08/21/2020 08:22:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #90000, processed 1871970 words, keeping 64 word types
08/21/2020 08:22:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #100000, processed 2044905 words, keeping 65 word types
08/21/2020 08:22:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #110000, processed 2235888 words, keeping 65 word types
08/21/2020 08:22:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #120000, processed 2450267 words, keeping 65 word types
08/21/2020 08:22:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #130000, processed 2631056 words, keeping 65 word types
08/21/2020 08:22:08 - INFO - gensim.models.word2vec -   PROGRESS:

save w2v to word2vecf/milestone_seq.128d
w2v CREATED_BY_seq


...

save w2v to word2vecf/CREATED_BY_seq.128d
w2v UPDATED_BY_seq


08/21/2020 08:25:08 - INFO - gensim.models.word2vec -   collecting all words and their counts
08/21/2020 08:25:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
08/21/2020 08:25:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #10000, processed 253166 words, keeping 204 word types
08/21/2020 08:25:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #20000, processed 463255 words, keeping 268 word types
08/21/2020 08:25:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #30000, processed 654538 words, keeping 309 word types
08/21/2020 08:25:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #40000, processed 859787 words, keeping 347 word types
08/21/2020 08:25:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #50000, processed 1090795 words, keeping 376 word types
08/21/2020 08:25:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #60000, processed 1300595 words, keeping

Sentence Num 720924


08/21/2020 08:25:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #70000, processed 1495792 words, keeping 414 word types
08/21/2020 08:25:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #80000, processed 1691255 words, keeping 434 word types
08/21/2020 08:25:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #90000, processed 1871970 words, keeping 451 word types
08/21/2020 08:25:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #100000, processed 2044905 words, keeping 464 word types
08/21/2020 08:25:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #110000, processed 2235888 words, keeping 471 word types
08/21/2020 08:25:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #120000, processed 2450267 words, keeping 480 word types
08/21/2020 08:25:08 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #130000, processed 2631056 words, keeping 489 word types
08/21/2020 08:25:08 - INFO - gensim.models.word2vec -   PR

save w2v to word2vecf/UPDATED_BY_seq.128d


In [ ]:
!rm -rf Bert/saved_models/

In [ ]:
!python Bert/run.py \
    --output_dir Bert/saved_models \
    --model_type roberta \
    --config_name roberta-base \
    --mlm \
    --block_size 128 \
    --per_gpu_train_batch_size 64 \
    --per_gpu_eval_batch_size 64 \
    --gradient_accumulation_steps 1 \
    --learning_rate 5e-5 \
    --weight_decay 0.01 \
    --adam_epsilon 1e-6 \
    --max_grad_norm 1.0 \
    --max_steps 100000 \
    --mlm_probability 0.2 \
    --warmup_steps 10000 \
    --logging_steps 50 \
    --save_steps 10000 \
    --evaluate_during_training \
    --save_total_limit 500 \
    --seed 123456 \
    --tensorboard_dir saved_models/tensorboard_logs 

2020-08-21 08:36:18.516252: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
08/21/2020 08:36:20 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
('milestone_seq', '40') 3
('milestone_seq', '64') 4
('milestone_seq', '6') 5
('milestone_seq', '31') 6
67
('CREATED_BY_seq', '54') 70
('CREATED_BY_seq', '404') 71
('CREATED_BY_seq', '155') 72
('CREATED_BY_seq', '209') 73
670
('UPDATED_BY_seq', '54') 740
('UPDATED_BY_seq', '404') 741
('UPDATED_BY_seq', '155') 742
('UPDATED_BY_seq', '209') 743
670
08/21/2020 08:36:42 - INFO - __main__ -   RobertaConfig {
  "_num_labels": 2,
  "architectures": null,
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob"

In [ ]:

 
# 此处为google drive中的文件路径,drive为之前指定的工作根目录，要加上
os.chdir("..")

In [ ]:
!cp -r saved_models/checkpoint-last bert-base
!cp saved_models/vocab.pkl bert-base/vocab.pkl

In [ ]:
!python Fusionlayer/mymain.py \
      --kfold=5 \
      --index=1 \
      --train_batch_size=256 \
      --eval_steps=5000 \
      --max_len_text=32\
      --epoch=10 \
      --lr=1e-5 \
      --output_path=Fusionlayer/saved_models \
      --pretrained_model_path=Bert/bert-base/checkpoint-last \
      --eval_batch_size=512 2>&1 | tee Fusionlayer/saved_models/log/i.txt

2020-09-02 06:36:24.839998: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
09/02/2020 06:36:26 - INFO - summarizer.preprocessing.cleaner -   'pattern' package not found; tag filters are not available for English
09/02/2020 06:36:26 - INFO - __main__ -   Argument Namespace(display_steps=100, epoch=10, eval_batch_size=512, eval_steps=5000, hidden_dropout_prob=0.1, hidden_size=384, index=1, kfold=5, lr=1e-05, max_grad_norm=1.0, max_len_text=32, num_hidden_layers=6, num_label=2, output_path='Fusionlayer/saved_models', pretrained_model_path='Bert/bert-base/checkpoint-last', seed=2020, train_batch_size=256, vocab_dim_v1=64, vocab_size_v1=500000)
09/02/2020 06:36:26 - INFO - gensim.models.utils_any2vec -   loading projection weights from word2vecf/milestone_seq.128d
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migrati

In [ ]:
import pandas as pd
res=pd.read_csv('result/submission_test_escalated_1_000165.csv')
res.fillna(0, inplace=True)

In [ ]:
pp=[]
for p in res['predict_escalated']:
    if p<=0:
        pp.append(0)
    elif p>1:
        pp.append(1)
    else:
        pp.append(p)
res['label_tensor']=pp
res[['label_tensor']].to_csv('result/tensor.txt',index=False)

In [ ]:
res['predict_escalated'].to_csv('result/pytorch1.csv',index=False)